In [1]:
import numpy as np
import cv2

In [10]:
def ssd_summary(proto_path, model_path, layer_type=False, 
                   layer_shape=False, get_flops=False, get_memory=False,
                   get_total_time=False, get_layer_time=False):
    # read the model
    net = cv2.dnn.readNetFromCaffe(proto_path, model_path)
    
    # check if the model is empty
    if net.empty():
        print("Net is empty")
        return
        
    # get the types of layers used in the model
    if layer_type:
        print("Net contains:")
        for t in net.getLayerTypes():
            print("\t{} layers of type {}".format(net.getLayersCount(t), t))
    
    # get the tensor shapes for the loaded model and specified input shape
    if layer_shape:
        layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 3, 224, 224])
        layers_names = net.getLayerNames()
        print("Net layers shapes: ")
        for l in range(len(layers_names)):
            in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
            print("  Layer {} has {} inputs and {} outputs".format(layers_names[l],
                                                                 in_num, out_num))
            for i in range(in_num):
                print("\tinput #{} has shape {}".format(i, in_shapes[l][i].flatten()))
            for i in range(out_num):
                print("\toutput #{} has shape {}".format(i, out_shapes[l][i].flatten()))
    
    # compute the number of FLOPs
    if get_flops:
        print("gflops: ", net.getFLOPS((1, 3, 224, 224)) * 1e-9)
    
    # report the amount of memory consumed for storing weights and intermediate tensors
    if get_memory:
        w, b = net.getMemoryConsumption((1, 3, 224, 224))
        print("weights (mb):", w * 1e-6, ", blobs (mb):", b * 1e-6)
    
    # perform a forward pass for a mock input:
    blob = cv2.dnn.blobFromImage(np.zeros((224, 224, 3), np.uint8), 1, (224, 224))
    net.setInput(blob)
    net.forward()
    
    # report the total time
    if get_total_time:
        total, timings = net.getPerfProfile()
        tick2ms = 1000 / cv2.getTickFrequency()
        print("inference (ms): {:2f}".format(total * tick2ms))

    # report the per layer inference time
    if get_layer_time:
        layer_names = net.getLayerNames()
        print("{: <30} {}".format("LAYER", "TIME (ms)"))
        for (i,t) in enumerate(timings):
            print("{: <30} {:.2f}".format(layer_names[i], t[0] * tick2ms))

In [14]:
model_path = "./model/MobileNetSSD_deploy.caffemodel"
proto_path = "./model/deploy.prototxt"

In [15]:
ssd_summary(proto_path, model_path, layer_type=True)

Net contains:
	3 layers of type Concat
	47 layers of type Convolution
	1 layers of type DetectionOutput
	13 layers of type Flatten
	12 layers of type Permute
	6 layers of type PriorBox
	35 layers of type ReLU
	1 layers of type Reshape
	1 layers of type Softmax
	1 layers of type __NetInputLayer__


In [16]:
ssd_summary(proto_path, model_path, layer_shape=True)

Net layers shapes: 
  Layer conv0 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer conv0/relu has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1  32 112 112]
  Layer conv1/dw has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer conv1/dw/relu has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer conv1 has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  32 112 112]
  Layer conv1/relu has 1 inputs and 1 outputs
	input #0 has shape [  1  32 112 112]
	output #0 has shape [  1  64 112 112]
  Layer conv2/dw has 1 inputs and 1 outputs
	input #0 has shape [  1  64 112 112]
	output #0 has shape [  1  64 112 112]
  Layer conv2/dw/relu has 1 inputs and 1 outputs
	input #0 has shape [  1  64 112 112]
	output #0 has shape [ 1 64 56 56]
  Layer 

	input #0 has shape [  1 512  14  14]
	input #1 has shape [  1   3 224 224]
	output #0 has shape [   1    2 2352]
  Layer conv13_mbox_loc_perm has 1 inputs and 1 outputs
	input #0 has shape [   1 1024    7    7]
	output #0 has shape [ 1 24  7  7]
  Layer conv13_mbox_loc_flat has 1 inputs and 1 outputs
	input #0 has shape [ 1 24  7  7]
	output #0 has shape [ 1  7  7 24]
  Layer conv13_mbox_conf has 1 inputs and 1 outputs
	input #0 has shape [ 1  7  7 24]
	output #0 has shape [   1 1176]
  Layer conv13_mbox_conf_perm has 1 inputs and 1 outputs
	input #0 has shape [   1 1024    7    7]
	output #0 has shape [  1 126   7   7]
  Layer conv13_mbox_conf_flat has 1 inputs and 1 outputs
	input #0 has shape [  1 126   7   7]
	output #0 has shape [  1   7   7 126]
  Layer conv13_mbox_priorbox has 1 inputs and 1 outputs
	input #0 has shape [  1   7   7 126]
	output #0 has shape [   1 6174]
  Layer conv14_2_mbox_loc has 2 inputs and 1 outputs
	input #0 has shape [   1 1024    7    7]
	input #1 has s